<a href="https://colab.research.google.com/github/duper203/upstage_cookbook/blob/main/PDF_to_Podcast_SolarPro.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Implementation of Notebook LM's PDF to Podcast

### Introduction

Inspired by [Notebook LM's](https://notebooklm.google/) podcast generation feature and a recent open source implementation of [Open Notebook LM](https://github.com/gabrielchua/open-notebooklm). In this cookbook we will implement a walkthrough of how you can build a PDF to podcast pipeline.

Given any PDF we will generate a conversation between a host and a guest discussing and explaining the contents of the PDF.

In doing so we will learn the following:
1. How we can use JSON mode and structured generation with open models like Solar Pro to extract a script for the Podcast given text from the PDF.
2. How we can use TTS models to bring this script to life as a conversation.


In [ ]:
!apt install -qU libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install -qU ffmpeg-python
!pip install -qU PyAudio
!pip install -qU cartesia #to access TTS model
!pip install -qU langchain-upstage langchain langchain_community

In [ ]:
import os

# Standard library imports
from pathlib import Path
from tempfile import NamedTemporaryFile
from typing import List, Literal, Tuple, Optional

# Third-party imports
from pydantic import BaseModel
from pypdf import PdfReader

# from together import Together
from cartesia import Cartesia
from pydantic import ValidationError

from google.colab import userdata


In [ ]:
# Paste in your Together AI and Cartesia API Key or load it
os.environ["CARTESIA_API_KEY"] = userdata.get("CARTESIA_API_KEY")

client_cartesia = Cartesia(api_key=os.environ.get("CARTESIA_API_KEY"))
os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
# client_together = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

## 1 - Load in PDF of Choice

In [ ]:
from langchain_upstage import UpstageDocumentParseLoader
import re
def get_PDF_text(file):
    text = ''
    loader = UpstageDocumentParseLoader(file, output_format='text')

    pages = loader.load()
    for page in pages:
      text += page.page_content

    text = re.sub(r'-\n', '', text)
    text = re.sub(r'\n', ' ', text)
    return text

In [ ]:
text = get_PDF_text('solar.pdf')
text

'SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-ScalingDahyun Kim→, Chanjun Park→†, Sanghoon Kim→†, Wonsung Lee→†, Wonho Song→ Yunsu Kim→, Hyeonwoo Kim→, Yungi Kim, Hyeonju Lee, Jihoo Kim Changbae Ahn, Seonghoon Yang, Sukyung Lee, Hyunbyung Park, Gyoungjin Gim Mikyoung Cha, Hwalsuk Lee†, Sunghun Kim†Upstage AI, South Korea{kdahyun, chanjun.park, limerobot, wonsung.lee, hwalsuk.lee, hunkim}@upstage.aiAbstractWe introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, demonstrating superior performance in various natural language processing (NLP) tasks. Inspired by recent efforts to efﬁciently up-scale LLMs, we present a method for scaling LLMs called depth up-scaling (DUS), which encompasses depthwise scaling and continued pretraining. In contrast to other LLM up-scaling methods that use mixture-of-experts, DUS does not require complex changes to train and inference efﬁciently. We show experimentally that DUS is simple yet effectiv

## 2 - Generate Script

### Define Dialoge Schema with Pydantic

We need a way of telling the LLM what the structure of the podcast script between the guest and host will look like. We will do this using `pydantic` models.

Below we define the required classes.

- The overall conversation consists of lines said by either the host or the guest. The `DialogueItem` class specifies the structure of these lines.
- The full script is a combination of multiple lines performed by the speakers, here we also include a scratchpad field to allow the LLM to ideate and brainstorm the overall flow of the script prior to actually generating the lines. The `Dialogue` class specifies this.

In [ ]:
# Adapted and modified from https://github.com/gabrielchua/open-notebooklm
SYSTEM_PROMPT = """
You are a world-class podcast producer tasked with transforming the provided input {text} into an engaging and informative podcast script. The input may be unstructured or messy, sourced from PDFs or web pages. Your goal is to extract the most interesting and insightful content for a compelling podcast discussion.

Ensure the response is in this format:
{{
"scratchpad":"<string>"
"name_of_guest": "<string>",
"dialogue": [
    {{
      "speaker": "Host (Jane)",
      "text": "<string>"
    }},
    {{
      "speaker": "Guest",
      "text": "<string>",
    }},
    ...
  ]
}}

# Steps to Follow:

1. **Analyze the Input:**
   Carefully examine the text, identifying key topics, points, and interesting facts or anecdotes that could drive an engaging podcast conversation. Disregard irrelevant information or formatting issues.

2. **Brainstorm Ideas:**
   In the `<scratchpad>`, creatively brainstorm ways to present the key points engagingly. Consider:
   - Analogies, storytelling techniques, or hypothetical scenarios to make content relatable
   - Ways to make complex topics accessible to a general audience
   - Thought-provoking questions to explore during the podcast
   - Creative approaches to fill any gaps in the information

3. **Craft the Dialogue:**
   Develop a natural, conversational flow between the host (Jane) and the guest speaker (the author or an expert on the topic). Incorporate:
   - The best ideas from your brainstorming session
   - Clear explanations of complex topics
   - An engaging and lively tone to captivate listeners
   - A balance of information and entertainment

   Rules for the dialogue:
   - The host (Jane) always initiates the conversation and interviews the guest. With 10 questions to ask based on the input
   - Include thoughtful questions from the host to guide the discussion
   - Incorporate natural speech patterns, including occasional verbal fillers (e.g., "Uhh", "Hmmm", "um," "well," "you know")
   - Allow for natural interruptions and back-and-forth between host and guest - this is very important to make the conversation feel authentic
   - Ensure the guest's responses are substantiated by the input text, avoiding unsupported claims
   - Maintain a PG-rated conversation appropriate for all audiences
   - Avoid any marketing or self-promotional content from the guest
   - The host concludes the conversation

4. **Summarize Key Insights:**
   Naturally weave a summary of key points into the closing part of the dialogue. This should feel like a casual conversation rather than a formal recap, reinforcing the main takeaways before signing off.

5. **Maintain Authenticity:**
   Throughout the script, strive for authenticity in the conversation. Include:
   - Moments of genuine curiosity or surprise from the host
   - Instances where the guest might briefly struggle to articulate a complex idea
   - Light-hearted moments or humor when appropriate
   - Brief personal anecdotes or examples that relate to the topic (within the bounds of the input text)

6. **Consider Pacing and Structure:**
   Ensure the dialogue has a natural ebb and flow:
   - Start with a strong hook to grab the listener's attention
   - Gradually build complexity as the conversation progresses
   - Include brief "breather" moments for listeners to absorb complex information
   - For complicated concepts, reasking similar questions framed from a different perspective is recommended
   - End on a high note, perhaps with a thought-provoking question or a call-to-action for listeners

IMPORTANT RULE
from the JSON response the "speaker" field it should only be "Host (Jane)" or "Guest".
Especially for "Guest" do not add other names to it just "Guest"

Remember: Always reply in valid JSON format, without code blocks. Begin directly with the JSON output.
"""

### Call the LLM to Generate Podcast Script

Below we call `Solar-Pro` to generate a script for our podcast. We will also be able to read it's `scratchpad` and see how it structured the overall conversation.

In [ ]:
class DialogueItem(BaseModel):
    """A single dialogue item."""

    speaker: Literal["Host (Jane)", "Guest"]
    text: str


class Dialogue(BaseModel):
    """The dialogue between the host and guest."""

    scratchpad: str
    name_of_guest: str
    dialogue: List[DialogueItem]

In [ ]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

def call_llm(system_prompt: str, text, dialogue_format):
    """Call the LLM with the given prompt and dialogue format."""
    llm = ChatUpstage(model='solar-pro-rc',extra_body={"response_format": {"type": "json"}})


    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{text}")
    ])

    chain = chat_prompt | llm

    response = chain.invoke({"text": text})
    return response


In [ ]:
response = call_llm(SYSTEM_PROMPT, text, Dialogue)

In [ ]:
response.content

'{\n    "scratchpad": "To create a captivating podcast script, we\'ll focus on the journey of developing SOLAR 10.7B, an impressive LLM with 10.7 billion parameters. We\'ll explore the innovative Depth Up-Scaling (DUS) approach, the challenges of scaling LLMs, and the significant performance improvements SOLAR 10.7B brings to various NLP tasks. We\'ll also delve into the instruction and alignment tuning stages, highlighting the creative use of datasets and the model\'s superior performance in following complex instructions. Lastly, we\'ll discuss the ethical considerations and the commitment to maintaining high ethical standards in the development and deployment of SOLAR 10.7B.",\n    "name_of_guest": "Dr. Chanjun Park",\n    "dialogue": [\n      {\n        "speaker": "Host (Jane)",\n        "text": "Welcome back to the \'AI Chronicles\' podcast! Today, we\'re diving into the fascinating world of Large Language Models with our guest, Dr. Chanjun Park, who\'s been instrumental in the de

In [ ]:
script = Dialogue.model_validate_json(response.content)

In [ ]:
script.dialogue

[DialogueItem(speaker='Host (Jane)', text="Welcome back to the 'AI Chronicles' podcast! Today, we're diving into the fascinating world of Large Language Models with our guest, Dr. Chanjun Park, who's been instrumental in the development of SOLAR 10.7B. Dr. Park, could you start by giving us an overview of SOLAR 10.7B and its significance in the field of Natural Language Processing?"),
 DialogueItem(speaker='Guest', text="Absolutely, Jane. SOLAR 10.7B is a Large Language Model with 10.7 billion parameters, which is a significant achievement in the NLP community. It's built on the innovative Depth Up-Scaling (DUS) approach, which allows us to scale up high-performance LLMs from smaller ones efficiently. This model has demonstrated superior performance in various NLP tasks, outperforming existing models like Llama 2 and Mistral 7B."),
 DialogueItem(speaker='Host (Jane)', text="That's impressive! Now, for our listeners who might not be familiar with DUS, could you explain how it works and 

## 3 - Generate Podcast Using TTS

Below we read through the script and parse choose the TTS voice depending on the speaker. We define a speaker and guest voice id.

We can loop through the lines in the script and generate them by a call to the TTS model with specific voice and lines configurations. The lines all appended to the same buffer and once the script finishes we write this out to a `wav` file, ready to be played.



In [ ]:
import subprocess
import ffmpeg

host_id = "694f9389-aac1-45b6-b726-9d9369183238" # Jane - host
guest_id = "a0e99841-438c-4a64-b679-ae501e7d6091" # Guest

model_id = "sonic-english" # The Sonic Cartesia model for English TTS

output_format = {
    "container": "raw",
    "encoding": "pcm_f32le",
    "sample_rate": 44100,
    }

# Set up a WebSocket connection.
ws = client_cartesia.tts.websocket()

# Open a file to write the raw PCM audio bytes to.
f = open("podcast.pcm", "wb")

# Generate and stream audio.
for line in script.dialogue:
    if line.speaker == "Guest":
        voice_id = guest_id
    else:
        voice_id = host_id

    for output in ws.send(
        model_id=model_id,
        transcript='-' + line.text, # the "-"" is to add a pause between speakers
        voice_id=voice_id,
        stream=True,
        output_format=output_format,
    ):
        buffer = output["audio"]  # buffer contains raw PCM audio bytes
        f.write(buffer)

# Close the connection to release resources
ws.close()
f.close()

# Convert the raw PCM bytes to a WAV file.
ffmpeg.input("podcast.pcm", format="f32le").output("podcast.wav").run()

# Play the file
subprocess.run(["ffplay", "-autoexit", "-nodisp", "podcast.wav"])

CompletedProcess(args=['ffplay', '-autoexit', '-nodisp', 'podcast.wav'], returncode=0)

In [ ]:
# Play the podcast
import IPython
IPython.display.Audio("podcast.wav")